In [1]:
def prepare_input_from_df(df, feature_cols, time_steps=60):

    df = df.copy()
    df = df[feature_cols].dropna()
    data = df.tail(time_steps)
    return data.values.reshape(1, time_steps, len(feature_cols)).astype(np.float32)


In [2]:
from collections import Counter

def agreger_predictions(predictions):

    compteur = Counter(predictions)
    top = compteur.most_common()
    if len(top) > 1 and top[0][1] == top[1][1]:
        return 99  # Indécis
    return top[0][0]


In [3]:
from tensorflow.keras.models import load_model
import numpy as np
import os

def predict_court_terme(df, feature_cols):

    model_paths = {
        "1h": "models/lstm_btc_10_1h.h5",
        "2h": "models/lstm_btc_10_2h.h5",
        "4h": "models/lstm_btc_10_4h.h5"
    }

    # model_paths = {
    #     "1w": "models/lstm_btc_MT_1w.h5",
    #     "12h": "models/lstm_btc_MT_12h.h5",
    #     "1d": "models/lstm_btc_MT_1d.h5"
    # }
    predictions = []

    for tf, path in model_paths.items():
        if not os.path.exists(path):
            print(f"Modèle manquant : {path}")
            continue

        model = load_model(path)
        input_data = prepare_input_from_df(df, feature_cols, time_steps=60)
        y_pred = model.predict(input_data)
        y_pred = int(np.round(y_pred).flatten()[0])  # -1, 0 ou 1
        predictions.append(y_pred)

        print(f"{tf} → prédiction : {y_pred}")

    if not predictions:
        return 99  # Aucun modèle dispo

    return agreger_predictions(predictions)


In [4]:
import pandas as pd

features = ['open','high','low','close','volume','rsi','rsima','macd','macd_diff','macd_signal',
            'stoch_%K','stoch_%D','lower_band','higher_band','ma_band','sma9','sma21',
            'Avg_day','Avg_corps','Avg_meches']

# Charger tes données récentes (par ex. df_live.csv)
df = pd.read_csv("database\data_with_indicators\\1w\ETH-USDT-USDT.csv")

# Appliquer la prédiction
prediction = predict_court_terme(df, features)
print("Prédiction court terme :", prediction)


<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Elodie\AppData\Local\Temp\ipykernel_1404\108119891.py:8: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv("database\data_with_indicators\\1w\ETH-USDT-USDT.csv")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


1h → prédiction : 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
2h → prédiction : 0


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
4h → prédiction : 0
Prédiction court terme : 0
